In [1]:
%load_ext autoreload
%autoreload 2

In [206]:
import requests
from config import *
from copy import deepcopy
import pandas as pd
from datetime import datetime
import polars as pl
from pathlib import Path




# Configs

In [ ]:
DATA_FOLDER = Path('data')


############## REGION - Get Payment
REGION_PARAMS = {
    "sdk":"1.28.3"
}
REGIONS_API = 'https://on-ramp-content.api.cx.metamask.io/regions'
REGIONS = [
    {
        "region":"de"
        , "country":"DE"
        , "fiat_currency": "/currencies/fiat/eur"
    },
    {
        "region":"us-fl"
        , "country":"US"
        , "fiat_currency": "/currencies/fiat/usd"
    },
    {
        "region":"gb"
        , "country":"UK"
        , "fiat_currency": "/currencies/fiat/gb"
    }
]

############## QUOTE
QUOTE_API = 'https://on-ramp.api.cx.metamask.io/providers/all/quote'
CRYPTOCURRENCIES = [
    {
        "name": "Ethereum Mainnet"
        , "id": "/currencies/crypto/1/0x0000000000000000000000000000000000000000"
    },
    {
        "name": "USDT (Ethereum)"
        , "id":  "/currencies/crypto/1/0xdac17f958d2ee523a2206206994597c13d831ec7"
    },
    {
        "name": "USDT (BNB Chain)"
        , "id":  "/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955"
    },
    
]

# Scrape Payment data for regions

In [4]:
def extract_payments_data(payment_dict:dict):
    payment_dict.pop('logo', None)
    payment_dict.pop('icons', None)
    return payment_dict

def add_region_payment_data(region_dict:str, region_api:str = REGIONS_API, query_params:dict = REGION_PARAMS):
    # using a deepcopy to avoid mutating the original input
    dict_copy = deepcopy(region_dict)
    response = requests.get(f"{region_api}/{dict_copy['region']}", params=query_params)
    try:
        data = response.json()
        payment_info = [extract_payments_data(x) for x in data["payments"]]
        dict_copy["payments"] = payment_info
        print("Add Payment data")
        return dict_copy
    except Exception as e:
        print("Error adding payment data")
        print(e)

In [11]:
REGIONS_WITH_PAYMENT = []
for region in REGIONS:
    print(f"extracting payment info for", region)
    region_it = add_region_payment_data(region)
    REGIONS_WITH_PAYMENT.append(region_it)    

extracting payment info for {'region': 'de', 'country': 'DE', 'fiat_currency': '/currencies/fiat/eur'}
Add Payment data
extracting payment info for {'region': 'us-fl', 'country': 'US', 'fiat_currency': '/currencies/fiat/usd'}
Add Payment data
extracting payment info for {'region': 'gb', 'country': 'UK', 'fiat_currency': '/currencies/fiat/gb'}
Add Payment data


# Generate Quote Scrapings

In [26]:
QUOTE_QUERY_PARAMS_BASE = []
for region in REGIONS_WITH_PAYMENT:
    for payment in region["payments"]:
        for crypto in CRYPTOCURRENCIES:
            QUOTE_QUERY_PARAMS_BASE.append({
                "country": region["country"]
                , "region": region["region"] 
                , "crypto_name": crypto["name"]
                , "payment_name": payment["name"]
                , "scrape_params": {
                    "regionId":f"/regions/{region['region']}"
                    , "paymentMethodId": payment["id"]
                    , "cryptoCurrencyId": crypto["id"]
                    , "fiatCurrencyId": region["fiat_currency"]
                }
                
            })

In [192]:
def get_success_quote_data(provider_json) -> dict:
    quote_data = {}
    quote_data["provider"] = provider_json.get("provider")
    
    quote_fields_to_scrape = [
          {"key":"amountIn", "default":None}
        , {"key":"amountOut", "default":None}
        , {"key":"exchangeRate", "default":None}
        , {"key":"networkFee", "default":None}
        , {"key":"providerFee", "default":None}
        , {"key":"bestRate", "default":False}
    ]
    
    # this is to keep the dict filling with the same pattern
    # for both error and success dict
    json_ref = provider_json
    if isinstance(provider_json.get("quote"), dict):
        json_ref = provider_json.get("quote")
    for field in quote_fields_to_scrape:
        quote_data[field["key"]] = json_ref.get(field["key"], field["default"])
    return quote_data

def _format_string(x):
    return x.replace("(","").replace(")", "").replace(" ", "-").lower()

def generate_filename(query_params:dict, amount, format="parquet"):
    keys_to_format = ["region", "crypto_name", "payment_name"]
    keys_formatted = [_format_string(query_params[x]) for x in keys_to_format]
    keys_formatted.append(str(amount))
    keys_formatted.append(query_params["scrape_params"]["fiatCurrencyId"].split("/")[-1])
    keys_formatted.insert(0, datetime.now().strftime("%Y%m%d_%H%M"))
    return f'{"__".join(keys_formatted)}.{format}'


def _scrape_one_quote(query_params: dict , amount: float, endpoint=QUOTE_API) -> dict:
    copy_params = deepcopy(query_params)
    copy_params["amount"] = amount
    copy_params["request_time"] = datetime.now()
    response = requests.get(endpoint, params=copy_params)
    print(response.status_code)
    # return response
    """ 
    I don't want to assume that a provider that is not in the "success" means it does not
    support the currency, it probably was just a error at this specific quoting
    Logging the error for completeness.
    It will also make coverage counting easier later on...
    """
    try:
        raw = response.json()
        all_quotes = []
        i=1
        for key in ["success", "error"]:
            if len(raw[key]) > 0:
                for provider_quote in raw[key]:
                    single_quote = get_success_quote_data(provider_quote)
                    single_quote["position"] = i if key == 'success' else None
                    single_quote["has_quote"] = True if key =='success' else False     
                    single_quote.update(copy_params)
                    all_quotes.append(single_quote)
                    if key == 'success':
                        i += 1
        return {"status": "success", "payload": all_quotes}
    except Exception as e: 
        print("Error scraping", copy_params)
        print(e)
        return {"status": "failure", "payload": response}

def scrape_one_quote(quote_params:dict, amount:float, endpoint:str, data_folder:Path):
    
    quote_schema = {
        "provider":pl.String
        , "amountIn": pl.Float64
        , "amountOut": pl.Float64
        , "exchangeRate": pl.Float64
        , "networkFee": pl.Float64
        , "providerFee": pl.Float64
        , "bestRate": pl.Boolean
        , "position": pl.Int32
        , "has_quote": pl.Boolean
        , "regionId": pl.String 
        , "paymentMethodId":pl.String 
        , "cryptoCurrencyId":pl.String
        , "fiatCurrencyId": pl.String 
        , "amount":pl.Float64
        , "request_time": pl.Datetime
    }   
    
    scrape_result = _scrape_one_quote(
        quote_params["scrape_params"]
        , amount=amount
        , endpoint=endpoint
    )
    
    if scrape_result["status"] == 'success':
        filename = generate_filename(quote_params, amount)
        (
            pl.DataFrame(scrape_result["payload"], schema = quote_schema)
            .write_parquet(data_folder / filename, compression="snappy")
        )


In [198]:
r = scrape_one_quote(scrape_params=quote_test["scrape_params"], amount = amount_test, endpoint=QUOTE_API)

200


# Bulk scraping

In [136]:
import numpy as np


def calculate_bin_midpoints(min_value=30, max_value=30000, n_bins=20):
    bins = [min_value * (max_value / min_value)**(i/n_bins) for i in range(n_bins + 1)]
    bins = [round(b) for b in bins]

    midpoints = []
    for i in range(len(bins) - 1):
        from_value = bins[i]
        to_value = bins[i + 1]
        midpoint = round((from_value + to_value) / 2)
        midpoints.append({"from": from_value, "midpoint": midpoint, "to": to_value})

    return midpoints

# Example usage:
bin_midpoints = calculate_bin_midpoints()
bin_midpoints

[{'from': 30, 'midpoint': 36, 'to': 42},
 {'from': 42, 'midpoint': 51, 'to': 60},
 {'from': 60, 'midpoint': 72, 'to': 85},
 {'from': 85, 'midpoint': 102, 'to': 119},
 {'from': 119, 'midpoint': 144, 'to': 169},
 {'from': 169, 'midpoint': 204, 'to': 238},
 {'from': 238, 'midpoint': 288, 'to': 337},
 {'from': 337, 'midpoint': 406, 'to': 475},
 {'from': 475, 'midpoint': 574, 'to': 672},
 {'from': 672, 'midpoint': 810, 'to': 949},
 {'from': 949, 'midpoint': 1144, 'to': 1340},
 {'from': 1340, 'midpoint': 1616, 'to': 1893},
 {'from': 1893, 'midpoint': 2284, 'to': 2674},
 {'from': 2674, 'midpoint': 3226, 'to': 3777},
 {'from': 3777, 'midpoint': 4556, 'to': 5335},
 {'from': 5335, 'midpoint': 6436, 'to': 7536},
 {'from': 7536, 'midpoint': 9090, 'to': 10644},
 {'from': 10644, 'midpoint': 12840, 'to': 15036},
 {'from': 15036, 'midpoint': 18137, 'to': 21238},
 {'from': 21238, 'midpoint': 25619, 'to': 30000}]

'https://on-ramp.api.cx.metamask.io/?country=US&region=us-fl&crypto_name=USDT+%28BNB+Chain%29&payment_name=Debit+or+Credit&scrape_params=regionId&scrape_params=paymentMethodId&scrape_params=cryptoCurrencyId&scrape_params=fiatCurrencyId&amount=100'

In [69]:
r.json()

{'product': 'On-Ramp',
 'service': 'On-Ramp API',
 'version': '1.40.1',
 'features': ['orders', 'analytics']}

In [66]:
QUOTE_QUERY_PARAMS_BASE[20]

{'country': 'US',
 'region': 'us-fl',
 'crypto_name': 'USDT (BNB Chain)',
 'payment_name': 'Debit or Credit',
 'scrape_params': {'regionId': '/regions/us-fl',
  'paymentMethodId': '/payments/debit-credit-card',
  'cryptoCurrencyId': '/currencies/crypto/56/0x55d398326f99059ff775485246999027b3197955',
  'fiatCurrencyId': '/currencies/fiat/usd'}}

In [64]:

r.json()



{'product': 'On-Ramp',
 'service': 'On-Ramp API',
 'version': '1.40.1',
 'features': ['orders', 'analytics']}